In [ ]:
import matplotlib
import pandas
import numpy as np
import matplotlib.pyplot as plt 
import os.path as path
import wget
import pandas as pd
from scipy.optimize import curve_fit
from mpl_toolkits.mplot3d import Axes3D